In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score as ARI

## pip install partition-igraph
import igraph as ig
import partition_igraph


## Simple block model graph

We generate a simple 1000-node graph with 10 communities of expected size 100 where
* p_in = 0.1, edge probability for pairs of nodes in the same community
* p_out = 0.025, edge probability for pairs of nodes in different communities


In [2]:
# Graph generation with 10 communities of expected size 100
P = np.full((10,10),.025)
np.fill_diagonal(P,.1)
g = ig.Graph.Preference(n=1000, type_dist=list(np.repeat(.1,10)),
                        pref_matrix=P.tolist(),attribute='class')
## the 'ground-truth' communities
tc = {k:v for k,v in enumerate(g.vs['class'])}

## Example using partition_igraph


In [3]:
## run Louvain and ECG (with Leiden):
Louvain = g.community_multilevel()
Ecg = g.community_ecg(ens_size=16, final='leiden', resolution=1.0)


In [4]:
## modularity (w.r.t. original weights for ECG)
print('Modularity with Louvain:',Louvain.modularity)
print('Modularity with ECG:',Ecg.modularity)


Modularity with Louvain: 0.19438468869260003
Modularity with ECG: 0.21222510890778576


In [5]:
## compute some graph-aware measure given ground truth communities

# for 'gam' partition are either 'igraph.clustering.VertexClustering' or 'dict'
print('Adjusted Graph-Aware Rand Index for Louvain:',g.gam(Louvain, tc))
print('Adjusted Graph-Aware Rand Index for ECG:',g.gam(Ecg, tc))

print('\nJaccard Graph-Aware for Louvain:',g.gam(Louvain, tc, method="jaccard", adjusted=False))
print('Jaccard Graph-Aware for ECG:',g.gam(Ecg, tc, method="jaccard", adjusted=False))


Adjusted Graph-Aware Rand Index for Louvain: 0.5838260201782259
Adjusted Graph-Aware Rand Index for ECG: 0.8300281824636697

Jaccard Graph-Aware for Louvain: 0.5628111273792094
Jaccard Graph-Aware for ECG: 0.7907469539113544


In [6]:
## compute the adjusted RAND index 
print("Adjusted non-Graph-Aware Rand Index for Louvain:",ARI(g.vs['class'], Louvain.membership))
print("Adjusted non-Graph-Aware Rand Index for ecg:",ARI(g.vs['class'], Ecg.membership))


Adjusted non-Graph-Aware Rand Index for Louvain: 0.45885976004310497
Adjusted non-Graph-Aware Rand Index for ecg: 0.7782542844437367
